# DSCI: Milestone I

*Authors (Group 3): Mitchie, Jianru, Aishwarya, Aditya*<br>
*Date: April 4th, 2021*

## Imports

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage

In [2]:
%load_ext rpy2.ipython
%load_ext memory_profiler

C:\Users\dengj\Miniconda3\envs\525\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


## 1. Downloading the data 

In [3]:
%pwd

'C:\\Users\\dengj\\ubc_mds_21\\ds_block6\\525_web_cloud_comp\\majacloud\\notebooks'

In [4]:
%cd C:\\Users\\dengj\\ubc_mds_21\\ds_block6\\525_web_cloud_comp\\majacloud\\notebooks

C:\Users\dengj\ubc_mds_21\ds_block6\525_web_cloud_comp\majacloud\notebooks


In [5]:
# Necessary metadata
article_id = 14096681 
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharerainfall/"

In [6]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

In [7]:
%%time

# get the file named data.zip
files_to_dl = ["data.zip"]  
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

Wall time: 1min 6s


In [8]:
%%time

# extract the file
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

Wall time: 22.2 s


## 2. Combine the data CSVs

### 2.1 Combine the data CSVs by `Pandas`

In [9]:
# get an idea how individual file looks like 
use_cols = ["time", "lat_min", "lat_max", "lon_min","lon_max", "rain (mm/day)"]
df1 = pd.read_csv("figsharerainfall/ACCESS-CM2_daily_rainfall_NSW.csv", usecols=use_cols,dtype={'TailNum': 'str'})
df1['model'] = "ACCESS-CM2"
df1.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1,1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
2,1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
3,1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
4,1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,ACCESS-CM2


In [10]:
%%time
%memit
# Shows time that regular python takes to merge file
# Join all data together using pandas
files = glob.glob('figsharerainfall/*_daily_rainfall_NSW.csv')
df = pd.concat((pd.read_csv(file, index_col=0, usecols=use_cols)
                .assign(model=re.findall(r'[^\/]+(?=\_d)', file)[0])
                for file in files)
              )
df.to_csv("figsharerainfall/combined_data.csv")

peak memory: 293.53 MiB, increment: 0.17 MiB
Wall time: 10min 51s


In [11]:
%%sh
du -sh figsharerainfall/combined_data.csv

6.7G	figsharerainfall/combined_data.csv


In [12]:
%%time
df = pd.read_csv("figsharerainfall/combined_data.csv")

Wall time: 2min 11s


In [13]:
print(df.shape)

(62467843, 7)


In [14]:
df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,figsharerainfall\ACCESS-CM2
1,1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,figsharerainfall\ACCESS-CM2
2,1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,figsharerainfall\ACCESS-CM2
3,1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,figsharerainfall\ACCESS-CM2
4,1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,figsharerainfall\ACCESS-CM2


### 2.2 Combine the data CSV by `dask`

In [ ]:
# Aish for dask approach

### 2.3 Summary of observations on run-time and memory usages in both options (THIS IS A DRAFT TO HAVE EVERYONE'S INPUT AND MAY COMPILE LATER)
Compare run times and memory usages of these options on different machines within your team, and summarize your observations in your milestone notebook.

- Mitchie:

- Jianru:

- Aishwarya:

- Aditya:

- Overall, TODO

## 3. Load the combined CSV to memory and perform a simple EDA

### 3.1 Load the entire data 

### 3.2 Load only the interested columns

### 3.3 Change `dtype` of the data

### 3.4 Load in chunks

In [ ]:
# Jianru py EDA 

### 3.5 Load using `dask`

### 3.6 Discussion of the observations on those approaches

- Mitchie:

- Jianru:

- Aishwarya:

- Aditya:

- Overall, TODO

## 4. Perform a simple EDA in R

In [ ]:
# Aditya R EDA

### 4.1 (One approach to transfer df from py to R)

### 4.2 Discussion on why you choose this approach over the others

- Mitchie:

- Jianru:

- Aishwarya:

- Aditya:

- Overall, TODO

## 5. Final Comments

In [15]:
import dask.dataframe as dd
import pandas as pd

In [16]:
%%time
%%memit

# show time that dask take to merge

ddf = dd.read_csv("figsharerainfall/*_daily_rainfall_NSW.csv", usecols=use_cols, assume_missing=True)
ddf.to_csv('figsharerainfall/combined_data_dask.csv', single_file=True)

peak memory: 3381.06 MiB, increment: 1557.35 MiB
Wall time: 9min 12s


In [17]:
ddf.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 6 entries, time to rain (mm/day)
dtypes: object(1), float64(5)

## EDA by python

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62467843 entries, 0 to 62467842
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   time           object 
 1   lat_min        float64
 2   lat_max        float64
 3   lon_min        float64
 4   lon_max        float64
 5   rain (mm/day)  float64
 6   model          object 
dtypes: float64(5), object(2)
memory usage: 3.3+ GB


In [42]:
f64_mem = df.iloc[:, 1:6].memory_usage().sum()/1e6
f32_mem = df.iloc[:, 1:6].astype('float32', errors='ignore').memory_usage().sum() / 1e6

In [43]:
print(f"Memory usage with float64: {f64_mem: .2f} MB")
print(f"Memory usage withdf float64: {f32_mem: .2f} MB")

Memory usage with float64:  2498.71 MB
Memory usage withdf float64:  1249.36 MB


In [44]:
%%time
%%memit

# example of how to load data in small chunks. We can pass chunksize as a parameter while using read_csv.

counts = pd.Series(dtype=int)

for chunk in pd.read_csv('figsharerainfall/combined_data.csv', chunksize = 10_000_000):
    counts = counts.add(chunk['rain (mm/day)'].value_counts(), fill_value = 0)
print(counts.astype(int))


-3.807373e-12    1
-1.879113e-12    1
-1.676546e-12    1
-1.633859e-12    1
-1.623300e-12    1
                ..
 2.862267e+02    1
 3.165474e+02    1
 3.219634e+02    1
 3.748413e+02    1
 4.329395e+02    1
Length: 41160284, dtype: int32
peak memory: 3181.25 MiB, increment: 3110.54 MiB
Wall time: 4min 21s


In [45]:
%%time
%%memit

# dask approach

# ddf = dd.read_csv('figsharerainfall/combined_data.csv')
ddf['rain (mm/day)'].compute()

peak memory: 2813.88 MiB, increment: 2423.91 MiB
Wall time: 40.2 s
